## Determine when a sampel is to sparse to use patch-based resampling

In [2]:
"""
Updated patch-selection and resampling decision toolkit for TreeCorr
Incorporates CV as a *sanity check only* and uses physics-driven criteria:
 - min objects per patch
 - min pair counts per patch (n*(n-1)/2)
 - fraction of empty patches
 - Kish effective N
 - patch angular radius relative to max correlation scale

Usage:
 - import functions from this file
 - call choose_patches_and_resampling(...) with your RA/Dec arrays and desired parameters
 - labels for chosen npatch are returned and can be used to build jackknife folds

Author: Nova (assistant)
"""

import numpy as np
from sklearn.cluster import KMeans
from collections import defaultdict
import math

# Optional: try to import healpy for healpix-based patching
try:
    import healpy as hp
    HAS_HEALPY = True
except Exception:
    HAS_HEALPY = False

# ----------------------------- Utilities -----------------------------

def sphere_to_cartesian(ra_deg, dec_deg):
    ra = np.deg2rad(ra_deg)
    dec = np.deg2rad(dec_deg)
    x = np.cos(dec) * np.cos(ra)
    y = np.cos(dec) * np.sin(ra)
    z = np.sin(dec)
    return np.vstack([x, y, z]).T


def angular_separation_deg(ra1_deg, dec1_deg, ra2_deg, dec2_deg):
    """Accurate angular separation in degrees between two points."""
    ra1, dec1, ra2, dec2 = map(np.deg2rad, (ra1_deg, dec1_deg, ra2_deg, dec2_deg))
    sin_d1, cos_d1 = np.sin(dec1), np.cos(dec1)
    sin_d2, cos_d2 = np.sin(dec2), np.cos(dec2)
    cos_dphi = np.cos(ra1 - ra2)
    cos_theta = sin_d1*sin_d2 + cos_d1*cos_d2*cos_dphi
    cos_theta = np.clip(cos_theta, -1.0, 1.0)
    return np.degrees(np.arccos(cos_theta))

# ----------------------------- Patching -----------------------------

def make_patches_kmeans(ra_deg, dec_deg, npatch, random_state=0):
    """k-means clustering on 3D unit sphere; returns labels 0..npatch-1"""
    xyz = sphere_to_cartesian(ra_deg, dec_deg)
    kmeans = KMeans(n_clusters=npatch, random_state=random_state, n_init=10)
    labels = kmeans.fit_predict(xyz)
    return labels


def make_patches_healpix(ra_deg, dec_deg, npatch):
    """Make patches by grouping healpix pixels. Requires healpy.
    We choose nside such that number of pixels ~ npatch, then group nearby pixels
    to form ~npatch patches. This is a simple heuristic and might be adapted.
    """
    if not HAS_HEALPY:
        raise RuntimeError("healpy not available")
    # choose nside with npix close to npatch
    target_npixels = npatch
    nside = 1
    while hp.nside2npix(nside) < target_npixels:
        nside *= 2
    pix = hp.ang2pix(nside, np.deg2rad(90-dec_deg), np.deg2rad(ra_deg), lonlat=False)
    # group contiguous pixels into patches by simple modular binning
    # (this is intentionally simple; user may want to replace with better grouping)
    uniq = np.unique(pix)
    # map pixel -> group by hashing
    pix_to_group = {p: (i % npatch) for i, p in enumerate(uniq)}
    labels = np.array([pix_to_group[p] for p in pix])
    return labels

# ----------------------------- Diagnostics -----------------------------

def patch_counts_from_labels(labels, npatch=None, weights=None):
    if npatch is None:
        npatch = labels.max()+1
    counts = np.bincount(labels, minlength=npatch)
    if weights is None:
        return counts
    # weighted counts: sum weights per patch
    w = np.zeros(npatch, dtype=float)
    for lab, ww in zip(labels, weights):
        w[lab] += ww
    return w


def patch_pair_counts(counts):
    """Approximate number of unique pairs inside each patch using n*(n-1)/2.
    For weighted samples this is only an approximation (could use sum_w*(sum_w-1)/2).
    Returns array of pair counts per patch.
    """
    n = np.asarray(counts)
    pairs = n*(n-1)/2
    return pairs


def patch_angular_radius_deg(labels, ra_deg, dec_deg, npatch=None):
    """Estimate patch angular radius (max angular separation from patch centroid) in degrees.
    This is a conservative estimate of patch size. If a patch has 1 object, radius=0.
    """
    if npatch is None:
        npatch = labels.max()+1
    radii = np.zeros(npatch, dtype=float)
    for p in range(npatch):
        mask = (labels == p)
        if mask.sum() == 0:
            radii[p] = 0.0
            continue
        ras = ra_deg[mask]
        decs = dec_deg[mask]
        # centroid in cartesian, then to (ra,dec)
        xyz = sphere_to_cartesian(ras, decs)
        centroid = xyz.mean(axis=0)
        # normalize centroid
        centroid = centroid / np.linalg.norm(centroid)
        # convert centroid to ra,dec
        cx, cy, cz = centroid
        dec_c = np.degrees(np.arcsin(cz))
        ra_c = np.degrees(np.arctan2(cy, cx)) % 360.0
        # compute max angular separation
        sep = angular_separation_deg(ra_c, dec_c, ras, decs)
        radii[p] = sep.max() if sep.size>0 else 0.0
    return radii


def kish_neff_from_counts(counts):
    counts = np.asarray(counts, dtype=float)
    s = counts.sum()
    s2 = (counts**2).sum()
    if s2 <= 0:
        return 0.0
    return (s*s)/s2


def patch_statistics(labels, ra_deg, dec_deg, weights=None, npatch=None):
    if npatch is None:
        npatch = labels.max()+1
    counts = patch_counts_from_labels(labels, npatch=npatch, weights=None if weights is None else weights)
    pairs = patch_pair_counts(counts)
    radii = patch_angular_radius_deg(labels, ra_deg, dec_deg, npatch=npatch)

    stats = {
        'npatch': int(npatch),
        'counts_min': int(counts.min()),
        'counts_median': float(np.median(counts)),
        'counts_mean': float(np.mean(counts)),
        'counts_std': float(np.std(counts, ddof=1)) if len(counts)>1 else 0.0,
        'cv': float(np.std(counts, ddof=1)/np.mean(counts)) if np.mean(counts)>0 else float('inf'),
        'frac_empty': float((counts==0).sum()/len(counts)),
        'pairs_min': float(pairs.min()),
        'pairs_median': float(np.median(pairs)),
        'pairs_mean': float(np.mean(pairs)),
        'kish_neff': float(kish_neff_from_counts(counts)),
        'patch_radius_deg_mean': float(radii.mean()),
        'patch_radius_deg_median': float(np.median(radii)),
        'patch_radius_deg_min': float(radii.min()),
        'patch_radius_deg_max': float(radii.max()),
    }
    return stats

# ----------------------------- Decision Logic -----------------------------

def choose_patches_and_resampling(ra_deg, dec_deg,
                                  npatch_candidates=None,
                                  method='kmeans',
                                  weights=None,
                                  # thresholds tuned for galaxy 2-pt work; adapt as needed
                                  min_per_patch=20,
                                  median_per_patch=30,
                                  frac_empty_max=0.10,
                                  pairs_min_per_patch=50,
                                  kish_neff_min=15,
                                  max_correlation_scale_deg=2.0,
                                  cv_warning=3.0,
                                  random_state=0):
    """Decide whether patch resampling is valid and recommend npatch & method.

    Key points of logic (aligned with user's request):
      - CV is only used as a WARNING (not a strict reject) unless it tracks emptiness.
      - Reject npatch if any of these are true:
         * min_per_patch not met
         * too many empty patches (frac_empty > frac_empty_max)
         * too few pairs per patch on average (pairs_min_per_patch)
         * kish neff too small
         * patch angular radius too small compared to the max correlation scale

    Returns dict with decision, stats per candidate, chosen labels when available, and warnings.
    """

    N = len(ra_deg)
    if npatch_candidates is None:
        # reasonable defaults: try a few values including small and large patch counts
        upper = max(10, min(500, max(10, N//10)))
        npatch_candidates = np.unique(np.concatenate([
            np.linspace(10, upper, num=8, dtype=int),
            np.array([20, 30, 50, 100, 200], dtype=int)
        ]))

    labels_cache = {}
    stats_cache = {}
    accepted = []
    warnings = defaultdict(list)

    for npatch in npatch_candidates:
        if npatch <= 1:
            continue
        # create labels
        if method == 'kmeans':
            labels = make_patches_kmeans(ra_deg, dec_deg, npatch, random_state=random_state)
        elif method == 'healpix':
            labels = make_patches_healpix(ra_deg, dec_deg, npatch)
        else:
            raise ValueError('unknown method')
        labels_cache[npatch] = labels
        st = patch_statistics(labels, ra_deg, dec_deg)
        stats_cache[npatch] = st

        # compute rejection conditions
        reject_reasons = []
        if st['counts_min'] < min_per_patch:
            reject_reasons.append(f"min_per_patch={st['counts_min']}<{min_per_patch}")
        if st['frac_empty'] > frac_empty_max:
            reject_reasons.append(f"frac_empty={st['frac_empty']:.3f}>{frac_empty_max}")
        if st['pairs_median'] < pairs_min_per_patch:
            reject_reasons.append(f"median_pairs={st['pairs_median']:.1f}<{pairs_min_per_patch}")
        if st['kish_neff'] < kish_neff_min:
            reject_reasons.append(f"kish_neff={st['kish_neff']:.1f}<{kish_neff_min}")
        # require patch radius to be at least the max correlation scale (preferably larger)
        if st['patch_radius_deg_median'] < max_correlation_scale_deg*0.5:
            # This is a softer check (0.5 factor) because patches are allowed to be smaller but ideally bigger
            reject_reasons.append(f"median_patch_radius={st['patch_radius_deg_median']:.2f}deg<~{max_correlation_scale_deg*0.5}deg")

        # CV only as a warning unless it's caused by emptiness (then already in reject reasons)
        if st['cv'] > cv_warning:
            warnings[npatch].append(f"High CV={st['cv']:.2f} (warning only) -- may reflect real LSS")

        if reject_reasons:
            warnings[npatch].extend(reject_reasons)
        else:
            accepted.append(npatch)

    decision = {
        'accepted': bool(accepted),
        'accepted_npatches': sorted(accepted),
        'chosen_npatch': max(accepted) if accepted else None,
        'method': method,
        'stats': stats_cache,
        'labels': labels_cache,
        'warnings': dict(warnings),
        'recommendation': None,
    }

    if not accepted:
        decision['recommendation'] = 'point_bootstrap'
        return decision

    # choose largest accepted npatch by default
    chosen = decision['chosen_npatch']
    decision['recommendation'] = 'patch_jackknife' if chosen >= 30 else 'patch_bootstrap'

    # If CV warnings exist for the chosen npatch, preserve as an advisory note
    if chosen in warnings and any('High CV' in w for w in warnings[chosen]):
        decision['warnings_for_choice'] = warnings[chosen]
    else:
        decision['warnings_for_choice'] = []

    # final note: return labels for chosen npatch for downstream jackknife
    decision['chosen_labels'] = labels_cache[chosen]

    return decision

# ----------------------------- Example use -----------------------------
if __name__ == '__main__':
    # small test with toy data
    import numpy as np
    rng = np.random.default_rng(123)
    # make an inhomogeneous toy survey: two overdensities + background
    N = 5000
    # background
    ra_bg = rng.uniform(0, 360, size=int(0.8*N))
    dec_bg = rng.uniform(-30, 30, size=int(0.8*N))
    # overdensity 1
    ra1 = rng.normal(180, 1, size=int(0.1*N))
    dec1 = rng.normal(0, 0.5, size=int(0.1*N))
    # overdensity 2
    ra2 = rng.normal(60, 0.8, size=int(0.1*N))
    dec2 = rng.normal(10, 0.4, size=int(0.1*N))
    ra = np.concatenate([ra_bg, ra1, ra2])
    dec = np.concatenate([dec_bg, dec1, dec2])

    decision = choose_patches_and_resampling(ra, dec, npatch_candidates=[20,50,100,200], random_state=1)
    print('Decision:')
    print(decision['recommendation'], 'chosen', decision['chosen_npatch'])
    print('Warnings (sample):', list(decision['warnings'].items())[:3])


# End of file


Decision:
patch_jackknife chosen 100
Warnings (sample): [(200, ['min_per_patch=10<20'])]


In [9]:
import numpy as np
import pandas as pd
import healpy as hp
import treecorr
import matplotlib.pyplot as plt

# ------------------------------------------------
# User parameters (from your snippet)
# ------------------------------------------------
GLADE_FILE = "../../data/GLADE_zhelio_lt0.1_dL_lt300.txt"
DEFL_FILE  = "../../data/deflexionZ2E510_R2_ra_dec_deg.txt"
FLUX_FILE  = "../../data/Auger/flux_a8_lb.dat"

PLOTS_DIR = "../../plots"

FWHM_DEG = 30.0
N_RAND_PER_GAL = 100
MIN_CZ = 1200
MAX_CZ = 4000
MK_CUT = -20.0
MAX_DEC = 45.0

TC_CONFIG = dict(
    min_sep=1,
    max_sep=65.,
    nbins=13,
    sep_units='degree',
    bin_type='Linear',
    brute=False,
    metric='Arc',
)

# ------------------------------------------------
# Helper functions (from your provided code)
# ------------------------------------------------

def read_GLADE(cols_to_use, cz_min=MIN_CZ, dL_max=200, filename=GLADE_FILE):
    from get_masks import get_milkyway_mask
    c = 299792.458

    chunksize = 200_000
    with open(filename, 'r') as f:
        header = f.readline().lstrip('#').split()

    chunks = []
    for chunk in pd.read_csv(filename, delim_whitespace=True, names=header,
                             usecols=cols_to_use, skiprows=1, chunksize=chunksize):
        # Apply filters
        chunk = chunk[chunk["dist_flag"] != 0]
        chunk["cz"] = c * chunk["z_cmb"]
        chunk = chunk[(chunk["cz"] > cz_min) & (chunk["d_L"] < dL_max)]
        chunk["M_K"] = chunk["K"] - 5*np.log10(chunk["d_L"]) - 25

        mask = np.isfinite(chunk["M_K"]) & np.isfinite(chunk["d_L"])
        mask &= np.isfinite(chunk["RA"]) & np.isfinite(chunk["Dec"])
        chunks.append(chunk.loc[mask])

    df = pd.concat(chunks, ignore_index=True)

    from get_masks import get_milkyway_mask
    df = df.iloc[get_milkyway_mask(df["RA"].values, df["Dec"].values)]
    df = df[df["Dec"] < MAX_DEC]
    return df


def generate_RanCat(N_total, dec_min=-90, dec_max=MAX_DEC):
    from get_masks import get_milkyway_mask

    # Generate in sin(dec)
    ra = np.random.uniform(0, 360, 3*N_total)
    sindec = np.random.uniform(np.sin(np.radians(dec_min)),
                               np.sin(np.radians(dec_max)),
                               3*N_total)
    dec = np.degrees(np.arcsin(sindec))

    mask = get_milkyway_mask(ra, dec)
    ra = ra[mask][:N_total]
    dec = dec[mask][:N_total]

    if len(ra) < N_total:
        raise ValueError("Not enough randoms after masking!")

    return ra, dec


# ------------------------------------------------
# Patch diagnostics
# ------------------------------------------------

def compute_patch_stats(patch_ids):
    """Return statistics for patch occupancy."""
    values, counts = np.unique(patch_ids, return_counts=True)
    num_patches = values.size

    min_ct = counts.min()
    frac_empty = np.mean(counts == 0)
    cv = counts.std() / counts.mean()

    kish_neff = (counts.sum())**2 / (counts**2).sum()

    info = dict(
        num_patches=num_patches,
        min=min_ct,
        median=np.median(counts),
        mean=np.mean(counts),
        std=np.std(counts),
        cv=cv,
        frac_empty=frac_empty,
        kish_neff=kish_neff,
    )
    return info, counts


def passes_thresholds(stats, Nbins):
    """
    Main logic for deciding if patching is usable.
    NEW criteria based on our theoretical discussion.
    """
    if stats["frac_empty"] > 0.10:
        return False, "too many empty patches"

    if stats["min"] < 20:
        return False, "patches too sparse (<20 galaxies)"

    if stats["kish_neff"] < 2 * Nbins:
        return False, "effective patch number too small"

    # CV only used for extreme cases
    if stats["cv"] > 2.5 and stats["min"] < 50:
        return False, "CV very high AND patch occupancy low"

    return True, "OK"


# ------------------------------------------------
# Run NK correlation with choice of resampling strategy
# ------------------------------------------------

def run_treecorr_NK(gxs, r_ra, r_dec, resampling_mode, npatch=None):
    import treecorr

    # galaxy catalog
    if npatch is None:
        cat_gal = treecorr.Catalog(
            ra=gxs["RA"], dec=gxs["Dec"],
            ra_units="degree", dec_units="degree",
            k=np.ones(len(gxs)),
        )
        cat_rnd = treecorr.Catalog(
            ra=r_ra, dec=r_dec,
            ra_units="degree", dec_units="degree",
            k=np.ones(len(r_ra)),
        )
    else:
        cat_gal = treecorr.Catalog(
            ra=gxs["RA"], dec=gxs["Dec"],
            ra_units="degree", dec_units="degree",
            k=np.ones(len(gxs)),
            npatch=npatch
        )
        cat_rnd = treecorr.Catalog(
            ra=r_ra, dec=r_dec,
            ra_units="degree", dec_units="degree",
            k=np.ones(len(r_ra)),
            npatch=npatch
        )

    config = dict(TC_CONFIG)
    config["var_method"] = resampling_mode

    nk = treecorr.NKCorrelation(config=config)
    nk.process(cat_gal, cat_rnd)

    print(f"Computed NK with resampling={resampling_mode}, npatch={npatch}")
    print("Cov shape:", None if nk.cov is None else nk.cov.shape)
    return nk


# ------------------------------------------------
# Master pipeline
# ------------------------------------------------

def pipeline():
    # ---- load data ----
    print("Loading GLADE...")
    gxs = read_GLADE(["RA", "Dec", "z_cmb", "K", "d_L", "dist_flag"])
    gxs = gxs[(gxs['cz'] < MAX_CZ) & (gxs['cz'] > MIN_CZ) &\
              (gxs['M_K'] < -24.) & (gxs['Dec'] < MAX_DEC)]
    print(f"  Loaded {len(gxs)} galaxies after cuts.")
    
    # Randoms
    print("Generating random catalog...")
    ra_rand, dec_rand = generate_RanCat(N_RAND_PER_GAL * len(gxs))

    # ---- try candidate patch numbers ----
    candidates = [20, 30, 40, 50, 60, 80, 100, 150]

    good_candidates = []

    for npatch in candidates:
        print(f"\nTesting npatch={npatch}")
        cat = treecorr.Catalog(
            ra=gxs["RA"], dec=gxs["Dec"],
            ra_units="degree", dec_units="degree",
            npatch=npatch
        )
        stats, counts = compute_patch_stats(cat.patch)
        ok, reason = passes_thresholds(stats, Nbins=TC_CONFIG["nbins"])

        print(f"  min={stats['min']}, median={stats['median']:.1f}, "
              f"mean={stats['mean']:.1f}, CV={stats['cv']:.2f}, "
              f"empty={stats['frac_empty']:.2f}, KishNeff={stats['kish_neff']:.1f}")
        print(f"  -> {reason}")

        if ok:
            good_candidates.append((npatch, stats))

    # ----------------------------------------------------------------
    # Decide the mode
    # ----------------------------------------------------------------
    if len(good_candidates) == 0:
        print("\n>>> No valid patching found. Switching to MANUAL point-by-point bootstrap.")
        mode = "manual"
        best_npatch = None
    else:
        # Choose the npatch with largest Kish effective number
        best_npatch = max(good_candidates, key=lambda t: t[1]["kish_neff"])[0]
        mode = "jackknife"
        print(f"\n>>> Selected npatch={best_npatch} using JACKKNIFE resampling")

    # ----------------------------------------------------------------
    # Run TreeCorr
    # ----------------------------------------------------------------
    nk = run_treecorr_NK(gxs, ra_rand, dec_rand, resampling_mode=mode, npatch=best_npatch)

    print("\nDone.")
    return nk


# ------------------------------------------------
# Execute
# ------------------------------------------------
nk = pipeline()


Loading GLADE...


/tmp/ipykernel_6096/1525763123.py:46: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  for chunk in pd.read_csv(filename, delim_whitespace=True, names=header,


  Loaded 648 galaxies after cuts.
Generating random catalog...

Testing npatch=20
  min=13, median=33.0, mean=32.4, CV=0.35, empty=0.00, KishNeff=17.8
  -> patches too sparse (<20 galaxies)

Testing npatch=30
  min=9, median=19.5, mean=21.6, CV=0.44, empty=0.00, KishNeff=25.2
  -> patches too sparse (<20 galaxies)

Testing npatch=40
  min=3, median=14.0, mean=16.2, CV=0.47, empty=0.00, KishNeff=32.9
  -> patches too sparse (<20 galaxies)

Testing npatch=50
  min=2, median=11.0, mean=13.0, CV=0.57, empty=0.00, KishNeff=37.7
  -> patches too sparse (<20 galaxies)

Testing npatch=60
  min=2, median=10.0, mean=10.8, CV=0.52, empty=0.00, KishNeff=47.2
  -> patches too sparse (<20 galaxies)

Testing npatch=80
  min=2, median=8.0, mean=8.1, CV=0.52, empty=0.00, KishNeff=62.7
  -> patches too sparse (<20 galaxies)

Testing npatch=100
  min=1, median=5.5, mean=6.5, CV=0.62, empty=0.00, KishNeff=72.0
  -> patches too sparse (<20 galaxies)

Testing npatch=150
  min=1, median=4.0, mean=4.4, CV=0.6

ValueError: Parameter var_method has invalid value manual.  Valid values are ['shot', 'jackknife', 'sample', 'bootstrap', 'marked_bootstrap'].